# **Data Cleaning Notebook**

Part of CRISP-DM **Data Understanding** and **Data Preparation**

## Objectives

* Data Exploration using ProfileReport
* Correlation and PPS Analysis
* Detailed evaluation of missing data
* Imputing strategies for missing data
* Split Train and Test Sets

## Inputs

* outputs/datasets/collection/house_prices.csv

## Outputs

* Cleaned Train and Test sets, saved under outputs/datasets/cleaned

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Raw Data

In [ ]:
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/house_prices.csv")
df.head(5)

---

# Data Exploration

## Pandas Profiling Report

First we want to get familiar with the dataset. Using ProfileReport we can look at the variable types, distribution, missing data levels, etc.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df=df, minimal=True)
profile.to_notebook_iframe()

NOTE

* The report shows that of the 24 variables, 4 are text values.
* Closer inspection shows that 2 of the numerical valiables ['OverallCond', 'OverallQual'] represent categories as well, making a total of 5 categorical variables.

## Correlation and PPS Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
sns.set(style="whitegrid")

def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman,
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.4, PPS_Threshold = 0.2,
                  figsize=(12,10), font_annot = 10)

# Data Cleaning

## Assessing Missing Data

Since there appears to be multiple variables with missing data, we filter them out and take a closer look.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
df[vars_with_missing_data].info()

In [ ]:
# Code from walkthrough project 02, data cleaning notebook

if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

NOTE

* 9 of 24 variables have missing values, which amounts to 37.5%.
* 2 variables have more than 50% zeros.
* Roughly 26% of cells are missing.

We use another more concise way to inspect the missing data by using a custom function called EvaluateMissingData.
For each variable it evaluates the number of missing data values and the overall percentage of missing data and returns a pandas dataframe.

In [ ]:
# Code from walkthrough project 02, data cleaning notebook

def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
        data={"RowsWithMissingData": missing_data_absolute,
              "PercentageOfDataset": missing_data_percentage,
              "DataType": df.dtypes}
    )
                       .sort_values(by=['PercentageOfDataset'], ascending=False)
                       .query("PercentageOfDataset > 0")
                      )
    return df_missing_data

In [ ]:
EvaluateMissingData(df)

NOTE

* 2 variables ['EnclosedPorch', 'WoodDeckSF'] have more than 85% missing values each.
* These variables may warrant dropping them because they will have little or no predictive power.

## Data Cleaning Proposal

* **Drop:** The dataset contains excessive missing values in the 'EnclosedPorch' and 'WoodDeckSF' columns, with over 80% of the data missing. These variables may not be useful for predictive modeling due to the lack of significant variation across houses.
* **Mean Imputation:** Suitable for 'LotFrontage' and 'BedroomAbvGr', as their distributions are close to normal.
* **Median Imputation:** Recommended for 'GarageYrBlt', as this feature is close to being normally distributed but may be skewed by extreme values.
* **Arbitrary Number Imputation:** Suitable for '2ndFlrSF' and 'MasVnrArea', as the mojority of values equal 0 the missing values can be reasonably imputed with 0, based on the assumption that the absence of data indicates the absence of a second floor or masonry veneer area.
* **Categorical Imputation:** Necessary for 'GarageFinish' and 'BsmtFinType1', as these are categorical variables that require a different imputation approach.

To clean the data and assess the effects of the cleaning we use the custom 'DataCleaningEffect()' function.

In [ ]:
# Code from Code Insititute's Feature Engine module

import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):
  """
  Function to visualize data cleaning effect
  """
  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Drop Variables

* Variables to drop: ['EnclosedPorch', 'WoodDeckSF']
* We create a separate data frame applying the imputation method
* We assess the effect of the data imputation

In [ ]:
from feature_engine.selection import DropFeatures

variables_method = ['EnclosedPorch', 'WoodDeckSF']
variables_method

imputer = DropFeatures(features_to_drop=variables_method)
df_method = imputer.fit_transform(df)
df_method.head(5)

### Mean Imputation

* Variables to impute the mean: ['LotFrontage' , 'BedroomAbvGr']
* We create a separate data frame applying the imputation method
* We assess the effect of the data imputation

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_method = ['LotFrontage' , 'BedroomAbvGr']
variables_method

imputer = MeanMedianImputer(imputation_method='mean', variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### Median Imputation

* Variables to impute the median: ['2ndFlrSF' , 'GarageYrBlt', 'MasVnrArea']
* We create a separate data frame applying the imputation method
* We assess the effect of the data imputation

In [ ]:

from feature_engine.imputation import MeanMedianImputer

variables_method = ['GarageYrBlt']
variables_method

imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### Arbitrary Number Imputation

* Variables to impute the arbitrary number 0: ['2ndFlrSF', 'MasVnrArea']
* We create a separate data frame applying the imputation method
* We assess the effect of the data imputation

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer

variables_method = ['2ndFlrSF', 'MasVnrArea']

imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### Categorical Imputation

* Variables to impute the category: ['BsmtFinType1', 'GarageFinish']
* We create a separate data frame applying the imputation method
* We assess the effect of the data imputation

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_method = ['BsmtFinType1', 'GarageFinish']
variables_method

imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

## Train and Test Split

We split our data into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

We now use the Train set to learn and then apply the imputation methods to both sets.

In [ ]:
# Drop Variables ['EnclosedPorch', 'WoodDeckSF']
variables_drop = ['EnclosedPorch', 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables_drop)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet), imputer.transform(TestSet)

# MeanMedianImputer 
variables_mean = ['LotFrontage' , 'BedroomAbvGr']
imputer = MeanMedianImputer(imputation_method='mean', variables=variables_mean)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

# MeanMedianImputer
variable_median = ['GarageYrBlt']
imputer = MeanMedianImputer(imputation_method='median', variables=variable_median)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

# ArbitraryNumberImputer 
variables_arbitrary = ['2ndFlrSF','MasVnrArea']
imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_arbitrary)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

# CategoricalImputer
variables_categorical = ['BsmtFinType1', 'GarageFinish']
imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_categorical)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

We check for missing data again, to ensure we have effectivly handled all missing data.

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

## Data Cleaning Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

data_cleaning_pipeline = Pipeline([
      ( 'drop',  DropFeatures(features_to_drop=['EnclosedPorch', 'WoodDeckSF']) ),
      ( 'mean',  MeanMedianImputer(imputation_method='mean',
                                     variables=['LotFrontage' , 'BedroomAbvGr']) ),
      ( 'median',  MeanMedianImputer(imputation_method='median',
                                     variables=['GarageYrBlt']) ),
      ( 'arbitrary', ArbitraryNumberImputer(arbitrary_number=0,
                                     variables=['2ndFlrSF','MasVnrArea']) ),
      ( 'categorical',  CategoricalImputer(imputation_method='missing',
                                     fill_value='Unf',
                                     variables=['GarageFinish' , 'BsmtFinType1']) )
])

df = data_cleaning_pipeline.fit_transform(df)
df.head()

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
try:
  # create here your folder
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

# Save the Train Set
TrainSet.to_csv("outputs/datasets/cleaned/train_set_cleaned.csv", index=False)

# Save the Test Set
TestSet.to_csv("outputs/datasets/cleaned/test_set_cleaned.csv", index=False)

# Save the cleaned Dataset
df.to_csv("outputs/datasets/cleaned/house_prices_cleaned.csv", index=False)